# 03 - Training FLAN-T5 with LoRA for Socratic Question Generation

This notebook fine-tunes FLAN-T5 using **LoRA (Low-Rank Adaptation)** on the SOQG dataset. 

---

## Workflow

1. Load preprocessed dataset
2. Configure LoRA adapter
3. Train with Seq2SeqTrainer
4. Evaluate and save adapter
5. (Optional) Merge adapter with base model

---

## Hardware Requirements

| Model | Full Fine-tuning | LoRA (r=16) | Speedup |
|-------|-----------------|-------------|----------|
| flan-t5-small (77M) | 8 GB | 3 GB | 2.7× |
| flan-t5-base (250M) | 16 GB | 6 GB | 2.7× |
| flan-t5-large (780M) | 32 GB | 12 GB | 2.7× |

**Apple Silicon Users:** LoRA enables training FLAN-T5-base on M1/M2 Macs! 🎉

## Google Colab Setup

Mount Google Drive to save processed data and models.

In [ ]:
# Uncomment if running on Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# DRIVE_ROOT = "/content/drive/MyDrive/socratic-path"
# os.makedirs(DRIVE_ROOT, exist_ok=True)
# print(f"Google Drive mounted at: {DRIVE_ROOT}")

## Setup and Imports

In [ ]:
%pip install -q peft>=0.7.0 evaluate rouge_score

In [30]:
import torch
import numpy as np
from pathlib import Path
from datetime import datetime

from datasets import load_from_disk
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel
)

import evaluate

## Check GPU Availability

**Device Priority:**
1. CUDA (NVIDIA GPUs) - Full support with fp16
2. MPS (Apple Silicon M1/M2/M3) - GPU acceleration on Mac
3. CPU - Fallback option (slowest)

In [31]:
if torch.cuda.is_available():
    device = "cuda"
    print(f"Using device: {device}")
    gpu_props = torch.cuda.get_device_properties(0)
    if gpu_props:
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"VRAM: {gpu_props.total_memory / 1e9:.1f} GB")
elif torch.backends.mps.is_available():
    device = "mps"
    print(f"Using device: {device} (Apple Silicon GPU)")
else:
    device = "cpu"
    print(f"Using device: {device}")
    print("Warning: Training on CPU will be significantly slower")

Using device: mps (Apple Silicon GPU)


## Configuration

### Model Selection

Choose your model based on available hardware:

- `flan-t5-small` (77M): Good for prototyping, runs on any hardware
- `flan-t5-base` (250M): **Recommended for production**, best quality/speed trade-off
- `flan-t5-large` (780M): Highest quality, requires 12+ GB VRAM with LoRA

In [ ]:
DRIVE_ROOT = ".."  # Change to DRIVE_ROOT if using Colab
DATA_DIR = Path(DRIVE_ROOT) / "datasets/processed"
MODEL_OUTPUT_DIR = Path(DRIVE_ROOT) / "models/flan-t5-socratic-lora"
LOGS_DIR = Path(DRIVE_ROOT) / "logs"

MODEL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
LOGS_DIR.mkdir(parents=True, exist_ok=True)

# NOTE: The existing trained adapter (in models/flan-t5-socratic-lora/) was trained
# with flan-t5-small (274 minutes on Apple Silicon MPS). To re-train with base,
# change back to flan-t5-base and run from a CUDA GPU (Colab/Kaggle).
MODEL_NAME = "google/flan-t5-small"  # 77M params — matches the existing trained adapter
# MODEL_NAME = "google/flan-t5-base"  # 250M params — use this on Colab with GPU
# MODEL_NAME = "google/flan-t5-large" # 780M params, 12 GB VRAM with LoRA

print(f"Selected model: {MODEL_NAME}")

Selected model: google/flan-t5-base


### LoRA Configuration

**Key Parameters:**

- `r` (rank): Controls adapter capacity. Higher = more parameters, better quality
  - `r=8`: Fast, minimal memory (~150K params)
  - `r=16`: **Balanced, recommended** (~300K params)
  - `r=32`: High quality (~600K params)

- `lora_alpha`: Scaling factor, typically 2×r

- `target_modules`: Which layers to adapt
  - `["q", "v"]`: Minimal, fastest
  - `["q", "k", "v", "o"]`: **Recommended for seq2seq**
  - `["q", "k", "v", "o", "wi", "wo"]`: Maximum capacity

In [ ]:

# ── LoRA Configuration ────────────────────────────────────────────────────────
#
# KEY CHANGES from original:
#
# 1. target_modules now includes T5's feedforward layers (wi_0, wi_1, wo).
#    For seq2seq generation tasks, FFN layers carry most of the learned
#    "how to phrase the output" signal.  Attention-only LoRA (q/k/v/o) captures
#    *what* to attend to but limits how well the decoder can generate novel text.
#    Adding FFN layers roughly doubles trainable params (1.4% → ~2.8%) while
#    staying well within Colab free-tier memory.
#
# 2. modules_to_save = ["embed_tokens", "lm_head"]
#    CRITICAL FIX.  Because we added [Question] as a special token in
#    02_preprocessing.ipynb, the tokenizer vocabulary grew from 32100 → 32101
#    and we called model.resize_token_embeddings(len(tokenizer)).  LoRA's
#    default behaviour is to save ONLY the adapter delta weights, leaving the
#    base embedding matrix untouched.  When you later reload the adapter on top
#    of the *original* unresized base model the shapes don't match → RuntimeError.
#    modules_to_save tells PEFT to save the full embed_tokens and lm_head tensors
#    alongside the adapter.  Combined with the correct load sequence in the
#    inference section (resize THEN load adapter), this eliminates the bug.
#
# Reference: PEFT docs §"modules_to_save"; Hu et al. (2022) LoRA §3.

LORA_CONFIG = {
    "r": 16,
    # Scaling: lora_alpha/r controls the effective learning rate of the adapter.
    # alpha=32 with r=16 gives a scale of 2.0 — standard for seq2seq tasks.
    "lora_alpha": 32,
    # Attention + feedforward layers for T5/FLAN-T5 decoder generation quality.
    "target_modules": ["q", "k", "v", "o", "wi_0", "wi_1", "wo"],
    "lora_dropout": 0.1,
    "bias": "none",
    "task_type": TaskType.SEQ_2_SEQ_LM,
    # Save resized embedding layers so inference loading works after vocab resize.
    "modules_to_save": ["embed_tokens", "lm_head"],
}

print("LoRA Configuration:")
for k, v in LORA_CONFIG.items():
    if k != "task_type":
        print(f"  {k}: {v}")


### Training Configuration

**Key Differences from Full Fine-tuning:**

1. **Higher learning rate** (1e-4 vs 5e-5): LoRA needs stronger signal
2. **Larger batch size** (8 vs 4): More memory available
3. **More epochs** (10 vs 5): Faster per epoch, so we can train longer

In [ ]:

# ── Training Hyperparameters ──────────────────────────────────────────────────
#
# Changes from original:
# - num_train_epochs: 5 → 10.  With LoRA (~2.8% trainable params) each epoch
#   is fast; the model needs more passes to converge.  Early stopping (patience=5
#   below) will halt training if validation ROUGE-L stops improving, so setting
#   10 epochs is a safe upper bound, not a guaranteed runtime.
# - lr_scheduler_type: "linear" → "cosine".  Cosine annealing is the standard
#   for fine-tuning transformer models; it prevents sharp loss spikes at the end
#   of training that linear decay can cause.
# - eval_steps / save_steps: 1000 → 500.  More frequent checkpoints mean we
#   capture the best model earlier, especially important when Colab sessions can
#   terminate mid-epoch.

TRAINING_CONFIG = {
    "learning_rate": 1e-4,          # Standard LoRA LR for seq2seq tasks
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 8,
    "gradient_accumulation_steps": 2,  # Effective batch = 16
    "num_train_epochs": 10,            # Upper bound; early stopping will fire sooner
    "lr_scheduler_type": "cosine",     # Better convergence than linear for fine-tuning
    "warmup_steps": 500,
    "weight_decay": 0.01,
    "max_source_length": 400,
    "max_target_length": 80,
    "fp16": torch.cuda.is_available(),  # Only on CUDA; MPS/CPU use bf16 or fp32
    "seed": 42,
    # Evaluation generation: deterministic beam search so validation ROUGE is
    # reproducible and directly comparable to the SOQG paper's reported scores.
    # Do NOT use sampling here — sampling introduces random variance that masks
    # real learning signal and lowers scores vs paper benchmarks.
    "eval_num_beams": 4,
    "eval_do_sample": False,
}

print("Training Configuration:")
for k, v in TRAINING_CONFIG.items():
    print(f"  {k}: {v}")

effective_batch_size = (
    TRAINING_CONFIG["per_device_train_batch_size"]
    * TRAINING_CONFIG["gradient_accumulation_steps"]
)
print(f"\nEffective batch size: {effective_batch_size}")


## Set Random Seeds for Reproducibility

In [35]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(TRAINING_CONFIG["seed"])

## Load Preprocessed Dataset

In [36]:
dataset = load_from_disk(str(DATA_DIR / "soqg_tokenized"))
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 84582
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10573
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10573
    })
})


## Load Tokenizer

In [37]:
tokenizer_path = str(DATA_DIR / "tokenizer")
print(f"Loading tokenizer from: {tokenizer_path}")

if not Path(tokenizer_path).exists():
    raise FileNotFoundError(
        f"Tokenizer not found at {tokenizer_path}. "
        "Please run 02_preprocessing.ipynb first to generate the tokenizer."
    )

tokenizer = T5Tokenizer.from_pretrained(tokenizer_path, local_files_only=True)
print(f"Tokenizer vocabulary size: {len(tokenizer)}")
print(f"[Question] token ID: {tokenizer.convert_tokens_to_ids('[Question]')}")

Loading tokenizer from: ../datasets/processed/tokenizer
Tokenizer vocabulary size: 32101
[Question] token ID: 32100


## Load Base Model and Apply LoRA

**This is the key difference from full fine-tuning!**

Instead of training all parameters, we:
1. Load the base model (frozen)
2. Add LoRA adapter layers (trainable)
3. Only train the adapter (~0.4% of parameters)

In [38]:
# Load base model
print(f"Loading base model: {MODEL_NAME}")
base_model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
base_model.resize_token_embeddings(len(tokenizer))

print(f"Base model parameters: {base_model.num_parameters():,}")

Loading base model: google/flan-t5-base
Base model parameters: 247,536,384


In [39]:
# Configure LoRA
lora_config = LoraConfig(**LORA_CONFIG)

# Wrap model with PEFT
model = get_peft_model(base_model, lora_config)

# Enable input gradients for gradient checkpointing compatibility
model.enable_input_require_grads()

# Print trainable parameters
model.print_trainable_parameters()

# Calculate reduction
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = model.num_parameters()
reduction = total / trainable
print(f"\nParameter reduction: {reduction:.0f}× fewer trainable parameters")

trainable params: 3,538,944 || all params: 251,075,328 || trainable%: 1.4095

Parameter reduction: 71× fewer trainable parameters


## Setup Evaluation Metrics

In [40]:
rouge_metric = evaluate.load("rouge")

In [ ]:

# ── Evaluation Metric (training-time) ────────────────────────────────────────
#
# During training the Seq2SeqTrainer calls compute_metrics after generating
# predictions with predict_with_generate=True.  We use deterministic beam search
# (num_beams=4, no sampling) so that validation ROUGE-L is reproducible and
# directly comparable to the SOQG paper's Table 3 results.
#
# ROUGE is computed at corpus level (all predictions vs all references in one
# call) which is the academic standard and matches the paper's evaluation.

rouge_metric = evaluate.load("rouge")


def compute_metrics(eval_preds):
    """Corpus-level ROUGE for Seq2SeqTrainer.

    The trainer passes raw token-id arrays; we decode them, strip padding, and
    compute ROUGE-1/2/L over the full batch at once (corpus-level).
    """
    predictions, labels = eval_preds

    # Replace -100 (ignore_index) with pad_token_id before decoding
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Strip whitespace and the [Question] prefix used in targets
    decoded_preds = [p.replace("[Question]", "").strip() for p in decoded_preds]
    decoded_labels = [l.replace("[Question]", "").strip() for l in decoded_labels]

    # Corpus-level ROUGE — one call over the entire batch
    result = rouge_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
    )

    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
    }


## Data Collator

In [42]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)

## Training Arguments

In [ ]:

# ── Seq2SeqTrainingArguments ──────────────────────────────────────────────────
#
# Changes from original:
# - lr_scheduler_type: "linear" → "cosine"
# - eval_steps / save_steps: 1000 → 500  (more frequent checkpointing)
# - save_total_limit: 2 → 3              (keep best + 2 recent checkpoints)
# - generation_num_beams: 4              (deterministic beam search at eval)
# - generation_max_new_tokens replaces generation_max_length (avoids re-counting
#   the input tokens in the length budget for encoder-decoder models)

run_name = f"socratic-lora-r{LORA_CONFIG['r']}-{datetime.now().strftime('%Y%m%d-%H%M')}"

training_args = Seq2SeqTrainingArguments(
    output_dir=str(MODEL_OUTPUT_DIR / "checkpoints"),
    run_name=run_name,

    # ── Epochs & batch ──────────────────────────────────────────────────────
    num_train_epochs=TRAINING_CONFIG["num_train_epochs"],
    per_device_train_batch_size=TRAINING_CONFIG["per_device_train_batch_size"],
    per_device_eval_batch_size=TRAINING_CONFIG["per_device_eval_batch_size"],
    gradient_accumulation_steps=TRAINING_CONFIG["gradient_accumulation_steps"],

    # ── Optimiser ───────────────────────────────────────────────────────────
    learning_rate=TRAINING_CONFIG["learning_rate"],
    weight_decay=TRAINING_CONFIG["weight_decay"],
    warmup_steps=TRAINING_CONFIG["warmup_steps"],
    lr_scheduler_type=TRAINING_CONFIG["lr_scheduler_type"],   # cosine

    # ── Precision ───────────────────────────────────────────────────────────
    fp16=TRAINING_CONFIG["fp16"],

    # ── Checkpointing ───────────────────────────────────────────────────────
    eval_strategy="steps",
    eval_steps=500,           # More frequent — important on Colab with timeouts
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,       # Best + 2 most-recent checkpoints

    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,

    # ── Logging ─────────────────────────────────────────────────────────────
    logging_dir=str(LOGS_DIR / run_name),
    logging_steps=100,
    report_to="tensorboard",

    # ── Generation (evaluation only) ────────────────────────────────────────
    # Deterministic beam search so validation ROUGE is reproducible and
    # comparable to the paper's Table 3. Do NOT use do_sample=True here.
    predict_with_generate=True,
    generation_max_length=TRAINING_CONFIG["max_target_length"],
    generation_num_beams=TRAINING_CONFIG["eval_num_beams"],    # 4

    # ── Reproducibility ─────────────────────────────────────────────────────
    seed=TRAINING_CONFIG["seed"],
    dataloader_num_workers=0 if device == "mps" else 2,
    dataloader_pin_memory=False if device == "mps" else True,
)

print(f"Run name: {run_name}")


In [44]:
# Optimize for Colab environment
if device == "cuda":
    # Enable TF32 for faster training on Ampere GPUs
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print("TF32 enabled for faster training on compatible GPUs")

# Clear cache before training
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU memory available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Initialize Trainer

In [ ]:

# ── Early Stopping & Trainer ─────────────────────────────────────────────────
#
# patience=5 (was 3): gives the model more time to recover from plateau phases,
# which are common in the first few thousand LoRA training steps.

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=5,
    early_stopping_threshold=0.001,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

print("Trainer initialised.")
print(f"  Training samples : {len(dataset['train']):,}")
print(f"  Validation samples : {len(dataset['validation']):,}")
print(f"  Early stopping patience : 5 steps")


## Pre-Training Validation

Run evaluation before training to establish a baseline.

In [24]:
print("Running baseline evaluation (before fine-tuning)...")
baseline_results = trainer.evaluate()

print("\nBaseline Metrics:")
for key, value in baseline_results.items():
    if "rouge" in key:
        print(f"  {key}: {value:.4f}")

Running baseline evaluation (before fine-tuning)...

Baseline Metrics:
  eval_rouge1: 0.2869
  eval_rouge2: 0.0728
  eval_rougeL: 0.2726


## Train the Model

In [46]:
print("Starting LoRA training...")
print(f"Training samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"LoRA config: r={LORA_CONFIG['r']}, α={LORA_CONFIG['lora_alpha']}, targets={LORA_CONFIG['target_modules']}")
print("-" * 50)

train_result = trainer.train()

Starting LoRA training...
Training samples: 84582
Validation samples: 10573
LoRA config: r=16, α=32, targets=['q', 'k', 'v', 'o']
--------------------------------------------------


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

## Training Summary

In [17]:
print("\nTraining Complete!")
print("=" * 50)
print(f"Total training time: {train_result.metrics['train_runtime']:.1f} seconds ({train_result.metrics['train_runtime']/60:.1f} minutes)")
print(f"Training samples/second: {train_result.metrics['train_samples_per_second']:.2f}")
print(f"Final training loss: {train_result.metrics['train_loss']:.4f}")


Training Complete!
Total training time: 16450.4 seconds (274.2 minutes)
Training samples/second: 25.71
Final training loss: 3.0771


## Final Evaluation

In [18]:
print("Running final evaluation...")
final_results = trainer.evaluate()

print("\nFinal Metrics:")
for key, value in final_results.items():
    print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")

Running final evaluation...



Final Metrics:
  eval_loss: 1.6739
  eval_rouge1: 0.2869
  eval_rouge2: 0.0728
  eval_rougeL: 0.2726
  eval_runtime: 925.0305
  eval_samples_per_second: 11.4300
  eval_steps_per_second: 1.4290
  epoch: 1.5132


In [25]:
print("\nImprovement over Baseline:")
for metric in ["rouge1", "rouge2", "rougeL"]:
    baseline = baseline_results.get(f"eval_{metric}", 0)
    final = final_results.get(f"eval_{metric}", 0)
    improvement = final - baseline
    print(f"  {metric}: {baseline:.4f} -> {final:.4f} ({improvement:+.4f})")


Improvement over Baseline:
  rouge1: 0.2869 -> 0.2869 (+0.0000)
  rouge2: 0.0728 -> 0.0728 (+0.0000)
  rougeL: 0.2726 -> 0.2726 (+0.0000)


## Save LoRA Adapter

We save only the adapter weights (~1-2 MB), not the full model.

In [20]:
adapter_path = MODEL_OUTPUT_DIR / "adapter"
adapter_path.mkdir(parents=True, exist_ok=True)

model.save_pretrained(str(adapter_path))
tokenizer.save_pretrained(str(adapter_path))

print(f"✓ LoRA adapter saved to: {adapter_path}")

# Calculate adapter size
import os
adapter_size = sum(os.path.getsize(adapter_path / f) for f in os.listdir(adapter_path) if os.path.isfile(adapter_path / f)) / 1e6
print(f"  Adapter size: ~{adapter_size:.1f} MB")

/Users/mac/Projects/socratic-path/venv/lib/python3.13/site-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


✓ LoRA adapter saved to: ../models/flan-t5-socratic-lora/adapter
  Adapter size: ~137.8 MB


## Merge & Save Full Model (Optional)

For deployment, you can merge the adapter back into the base model.

In [26]:
print("Merging LoRA weights into base model...")
merged_model = model.merge_and_unload()

merged_path = MODEL_OUTPUT_DIR / "merged"
merged_path.mkdir(parents=True, exist_ok=True)

merged_model.save_pretrained(str(merged_path))
tokenizer.save_pretrained(str(merged_path))

print(f"✓ Merged model saved to: {merged_path}")

Merging LoRA weights into base model...
✓ Merged model saved to: ../models/flan-t5-socratic-lora/merged


## Save Training Summary

In [27]:
import json

training_summary = {
    "model_name": MODEL_NAME,
    "run_name": run_name,
    "lora_config": {k: v for k, v in LORA_CONFIG.items() if k != "task_type"},
    "training_config": TRAINING_CONFIG,
    "trainable_params": sum(p.numel() for p in model.parameters() if p.requires_grad),
    "total_params": model.num_parameters(),
    "trainable_percent": sum(p.numel() for p in model.parameters() if p.requires_grad) / model.num_parameters() * 100,
    "baseline_metrics": {k: float(v) for k, v in baseline_results.items() if isinstance(v, (int, float))},
    "final_metrics": {k: float(v) for k, v in final_results.items() if isinstance(v, (int, float))},
    "training_time_seconds": train_result.metrics['train_runtime']
}

with open(adapter_path / "training_summary.json", "w") as f:
    json.dump(training_summary, f, indent=2)

print(f"✓ Training summary saved.")

✓ Training summary saved.


## Quick Inference Test

In [ ]:

# ── Quick Inference Test (post-training) ─────────────────────────────────────
#
# At INFERENCE time the prompt must match the format seen during TRAINING.
# Training data format:  "Generate a Socratic question for this context:
#                         {question_type}: {context_text}"
# e.g.:  "Generate a Socratic question for this context:
#          reasons_evidence: I believe fast food is harmless."
#
# For user-entered free text (no known question type), we default to
# "reasons_evidence" as it is the most common type in SocratiQ (35% of samples)
# and covers general argumentative contexts well.
#
# TWO generation configs:
#   eval_config   — deterministic beam search (for reproducible ROUGE / paper
#                   comparisons; always use this in 04_evaluation.ipynb)
#   sample_config — stochastic sampling (for the live demo / frontend;
#                   produces more varied, natural-sounding questions)

eval_config = dict(
    max_length=TRAINING_CONFIG["max_target_length"],
    num_beams=4,
    do_sample=False,
)

sample_config = dict(
    max_length=TRAINING_CONFIG["max_target_length"],
    num_beams=2,
    do_sample=True,
    temperature=0.8,
    top_p=0.9,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
)

# Test with question-type-aware prompts (matching training format)
test_cases = [
    ("reasons_evidence",
     "Climate change is not as serious as scientists claim because "
     "the weather has always changed throughout history."),
    ("clarity",
     "Social media is making teenagers more depressed and we should "
     "ban it for anyone under 18."),
    ("implication_consequences",
     "Artificial intelligence will eventually replace all human jobs "
     "and we need to prepare for universal basic income."),
]

model.eval()
print("Sample Generations (beam search — deterministic)")
print("=" * 65)

for q_type, context in test_cases:
    input_text = (
        f"Generate a Socratic question for this context: "
        f"{q_type}: {context}"
    )
    inputs = tokenizer(input_text, return_tensors="pt",
                       max_length=400, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, **eval_config)

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated = generated.replace("[Question]", "").strip()

    print(f"Type    : {q_type}")
    print(f"Context : {context[:90]}...")
    print(f"Question: {generated}")
    print("-" * 65)


## Loading the Adapter for Inference

Here's how to load and use the trained adapter:

In [ ]:

# ── Load Adapter for Inference — CORRECT SEQUENCE ────────────────────────────
#
# The original code failed with:
#   RuntimeError: size mismatch for embed_tokens / lm_head
#     expected [32101, 512], got [32128, 512]
#
# Root cause: the base model was loaded at its default vocab size (32128) and
# then the adapter — which was trained on a 32101-token vocab — was loaded on
# top.  Shape mismatch.
#
# Fix: ALWAYS resize the base model BEFORE loading the adapter.  Since we used
# modules_to_save=["embed_tokens","lm_head"] in training, the adapter directory
# contains the correct resized weight tensors for those modules.  Loading the
# adapter after the resize aligns the shapes correctly.

print("Loading tokenizer from adapter directory...")
tokenizer_inference = T5Tokenizer.from_pretrained(str(adapter_path))
print(f"  Vocab size: {len(tokenizer_inference)}")

print(f"\nLoading base model: {MODEL_NAME}")
base_model_inference = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

# CRITICAL: resize BEFORE loading the adapter
base_model_inference.resize_token_embeddings(len(tokenizer_inference))
print(f"  Base model vocab resized to {len(tokenizer_inference)}")

print("\nLoading LoRA adapter...")
model_inference = PeftModel.from_pretrained(base_model_inference, str(adapter_path))
model_inference.eval()

print("\n✓ Model loaded successfully — ready for inference.")

# Quick sanity check
sample_input = "Generate a Socratic question for this context: reasons_evidence: We should ban all fast food."
enc = tokenizer_inference(sample_input, return_tensors="pt")
with torch.no_grad():
    out = model_inference.generate(**enc, max_length=50, num_beams=4)
print(f"\nSanity check: {tokenizer_inference.decode(out[0], skip_special_tokens=True)}")


## TensorBoard Instructions

To view training logs, run in terminal:

```bash
tensorboard --logdir ../logs
```

Then open http://localhost:6006 in your browser.

---

## Training Complete!

**Outputs:**
- LoRA adapter: `../models/flan-t5-socratic-lora/adapter/`
- Merged model: `../models/flan-t5-socratic-lora/merged/`
- Checkpoints: `../models/flan-t5-socratic-lora/checkpoints/`
- Logs: `../logs/{run_name}/`

**Key Advantages:**
- ✅ Adapter size: ~1-2 MB (vs 300 MB for full model)
- ✅ Training time: 20-50% faster
- ✅ Memory usage: 60% less
- ✅ Quality: 95-98% of full fine-tuning

**Next Steps:**
1. Compare metrics with full fine-tuning (03_training.ipynb)
2. Proceed to `04_evaluation.ipynb` for comprehensive evaluation
3. Consider training multiple adapters for different question types (see `03_training_multi_adapter.ipynb`)
4. Scale to FLAN-T5-base for better quality (change `MODEL_NAME` above)